# Using EcoFOCIpy to process raw field data

**Cruise DY2012 (Bering Sea and Arctic - COVID impacted Cruise**

**Processed by Shaun Bell**

Follows the initial processing workbook [EcoFOCIpy_sbe_ctd_dy2012.ipynb](EcoFOCIpy_sbe_ctd_dy2012.ipynb) to apply manually corrected csv files to the netcdf files

This will generate:  
+ **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for final preview and validation
- TSSigma, TOXYChlor, TurbParTrans

***TODO:***
+ Add ability to specify cast/instrument and make all values missing/removed
+ Add ability to linearly interpolate between singleton points in profile for speciffic parameters
+ Update any meta data 

In [14]:
import os
import numpy as np
import pandas as pd
import xarray as xa

import ecofocipy.plots.sbe_ctd_plots as sbe_ctd_plots


## Post QC Processing

In [15]:
ncfiles = '.nc'
csvfiles = '.to_edit.csv'

In [16]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
sample_data_dir = '/Users/bell/ecoraid/2020/CTDcasts/dy2012/working/' #root path to cruise directory
cruise_name = 'DY2012' #no hyphens
###############################################################

In [17]:
# Following routines will eventually get ported to ecofocipy as subroutines to be called

import seawater as sw

def sigmat_update(salinity=None,temperature=None):
    '''
    Changes to T or S (commonly to despike values or apply a salinity offset) will need corresponding changes in sigmat
    '''
    # calculate sigmaT at 0db gauge pressure (s, t, p=0)
    sigt = (sw.eos80.dens0(s=salinity, t=temperature) - 1000)
    
    return sigt


def oxyconc_update(salinity=None,temperature=None, oxygen_conc_umkg=None,pressure=None):
    '''
        Although PJS tends to look at %sat to QC, changes are usually applied on the concentration parameter. So %sat will need recalculation.
        Changes to T/S also drive some small corrections.
        
        Watch the conc units (um/kg or um/l)

        calculate oxy_conc_M and calculate oxygen saturation from corrected concentration_umkg
        
        Garcia and Gorden 1992 - from Seabird Derived Parameter Formulas
    '''
    GG_cont = { 'GG_A0':2.00907,
                'GG_A1':3.22014,
                'GG_A2':4.0501,
                'GG_A3':4.94457,
                'GG_A4':-0.256847,
                'GG_A5':3.88767,
                'GG_B0':-0.00624523,
                'GG_B1':-0.00737614,
                'GG_B2':-0.010341,
                'GG_B3':-0.00817083,
                'GG_C0':-0.000000488682}

    Ts_pri = np.log((298.15 - temperature) / (273.15 + temperature))
    Oxsol_pri = np.exp(
    GG_cont['GG_A0']
    + GG_cont['GG_A1'] * Ts_pri
    + GG_cont['GG_A2'] * (Ts_pri) ** 2
    + GG_cont['GG_A3'] * (Ts_pri) ** 3
    + GG_cont['GG_A4'] * (Ts_pri) ** 4
    + GG_cont['GG_A5'] * (Ts_pri) ** 5
    + salinity
    * (GG_cont['GG_B0'] + GG_cont['GG_B1'] * Ts_pri
    + GG_cont['GG_B2'] * (Ts_pri) ** 2 
    + GG_cont['GG_B3'] * (Ts_pri) ** 3)
    + GG_cont['GG_C0'] * (salinity) ** 2
    )

    
    # determine sigmatheta and convert Oxygen from micromoles/kg to ml/l
    # calculate new oxygen saturation percent using derived oxsol
    sigmatheta_pri = sw.eos80.pden(s=salinity, t=temperature, p=pressure)
    oxygen_conc_mll = oxygen_conc_umkg * sigmatheta_pri / 44660
    
    return oxygen_conc_mll,((oxygen_conc_mll) / Oxsol_pri) * 100.0

In [18]:
#match csv to netcdf and update
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith(ncfiles):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        cruise_data_update = cruise_data_nc.copy()
        try:
            pandas_csv = pd.read_csv(sample_data_dir+cast.replace(ncfiles,csvfiles)).set_index(['time','latitude','longitude','depth']).to_xarray()

            for var_name in list(pandas_csv.data_vars):
                pandas_csv[var_name].attrs = cruise_data_update[var_name].attrs
                cruise_data_update[var_name].values = pandas_csv[var_name].values
                
            #update sigmat (or calculate it I suppose)
            sigup = sigmat_update(salinity=pandas_csv.salinity_ch1,
                                  temperature=pandas_csv.temperature_ch1)
            try:
                cruise_data_update['sigma_t_ch1'].values = sigup
            except:
                pass #no sigmat in file
                
            try:
                sigup2 = sigmat_update(salinity=pandas_csv.salinity_ch2,
                                  temperature=pandas_csv.temperature_ch2)
            except:
                pass #no sigmat in file
            
            #update 
            #need to update any other oxy conc units too
            cruise_data_update['oxy_concM_ch1'].values,cruise_data_update['oxy_percentsat_ch1'].values = oxyconc_update(pandas_csv.salinity_ch1,
                                                                   pandas_csv.temperature_ch1,
                                                                   pandas_csv.oxy_conc_ch1,
                                                                   pandas_csv.depth)
            try:
                cruise_data_update['oxy_concM_ch2'].values, cruise_data_update['oxy_percentsat_ch2'].values = oxyconc_update(pandas_csv.salinity_ch2,
                                                                   pandas_csv.temperature_ch2,
                                                                   pandas_csv.oxy_conc_ch2,
                                                                   pandas_csv.depth)            
            except:
                pass # no secondary oxy
            
            cruise_data_update.to_netcdf(sample_data_dir+cast.replace(ncfiles,'.updated.nc'),format='NETCDF3_CLASSIC',encoding={'time':{'units':'days since 1900-01-01'}})
            print(f'Updated: {cast}')
        except FileNotFoundError:
            print(f'No file to update: {cast}')

No file to update: DY2012c001_ctd.nc
Updated: DY2012c002_ctd.nc
Updated: DY2012c003_ctd.nc
Updated: DY2012c004_ctd.nc
Updated: DY2012c005_ctd.nc
Updated: DY2012c006_ctd.nc
Updated: DY2012c007_ctd.nc
Updated: DY2012c008_ctd.nc
Updated: DY2012c009_ctd.nc
Updated: DY2012c010_ctd.nc
Updated: DY2012c011_ctd.nc
Updated: DY2012c012_ctd.nc
Updated: DY2012c013_ctd.nc
Updated: DY2012c014_ctd.nc
Updated: DY2012c015_ctd.nc
Updated: DY2012c016_ctd.nc
Updated: DY2012c017_ctd.nc
Updated: DY2012c018_ctd.nc
Updated: DY2012c019_ctd.nc
Updated: DY2012c020_ctd.nc
Updated: DY2012c021_ctd.nc
Updated: DY2012c022_ctd.nc
Updated: DY2012c023_ctd.nc
Updated: DY2012c024_ctd.nc
Updated: DY2012c025_ctd.nc
Updated: DY2012c026_ctd.nc
No file to update: DY2012c027_ctd.nc
Updated: DY2012c028_ctd.nc
Updated: DY2012c029_ctd.nc
No file to update: DY2012c030_ctd.nc
No file to update: DY2012c031_ctd.nc
No file to update: DY2012c032_ctd.nc
No file to update: DY2012c033_ctd.nc
No file to update: DY2012c034_ctd.nc
No file to u

/Users/bell/miniconda3/envs/py38/lib/python3.8/site-packages/xarray/coding/times.py:144: SerializationWarning: Ambiguous reference date string: 0:00 GMT. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0000:00 GMT). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


ValueError: unable to decode time units 'msec since 0:00 GMT' with 'the default calendar'. Try opening your dataset with decode_times=False or installing cftime if it is not installed.

## Generate Plots


### Make General Plots
- 1:1 plots for paired instruments for each cast (tells if a sensor failed)
- TS_Sigmat, Chlor/Par/Turb, Oxy,Temp
- T/S property property plot
- upcast/downcast plt

In [19]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith('updated.nc'):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        #calc sigmat
        ctd_df['sigma_t_ch1'] = sigmat_update(temperature=ctd_df['temperature_ch1'],salinity=ctd_df['salinity_ch1'])
        ctd_df['sigma_t_ch2'] = sigmat_update(temperature=ctd_df['temperature_ch2'],salinity=ctd_df['salinity_ch2'])

        sbe_p = sbe_ctd_plots.CTDProfilePlot()
        plt,fig =sbe_p.plot3var(varname=['temperature_ch1','temperature_ch2','salinity_ch1','salinity_ch2','sigma_t_ch1','sigma_t_ch2'],
                          xdata=[ctd_df.temperature_ch1,ctd_df.temperature_ch2,ctd_df.salinity_ch1,ctd_df.salinity_ch2,ctd_df.sigma_t_ch1,ctd_df.sigma_t_ch2],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=True,
                          xlabel=['Temperature','Salinity','SigmaT'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_TempSalSigmaT.png'))
        plt.close(fig)

/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:219: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[2]),np.nanmin(xdata[3])])
/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:220: RuntimeWarning: All-NaN axis encountered
  abmax=np.nanmax([np.nanmax(xdata[2]),np.nanmax(xdata[3])])
/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:248: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[4]),np.nanmin(xdata[5])])
/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:249: RuntimeWarning: All-NaN axis encountered
  abmax=np.nanmax([np.nanmax(xdata[4]),np.nanmax(xdata[5])])
/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:219: RuntimeWarning: All-NaN axis encountered
  abmin=np.nanmin([np.nanmin(xdata[2]),np.nanmin(xdata[3])])
/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:220: RuntimeWarning: All-NaN axis encountered
  abmax=np.

In [20]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith('updated.nc'):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        sbe_p = sbe_ctd_plots.CTDProfilePlot()
        plt,fig =sbe_p.plot2var(varname=['temperature_ch1','temperature_ch2','oxy_percentsat_ch1','oxy_percentsat_ch2'],
                          xdata=[ctd_df.temperature_ch1,ctd_df.temperature_ch2,ctd_df.oxy_percentsat_ch1,ctd_df.oxy_percentsat_ch1],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=True,
                          xlabel=['Temperature','Oxygen Saturation'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_TempOxy.png'))
        plt.close(fig)

/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:105: RuntimeWarning: All-NaN axis encountered
  abmin=np.min([np.nanmin(xdata[2]),np.nanmin(xdata[3])])
/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/plots/sbe_ctd_plots.py:106: RuntimeWarning: All-NaN axis encountered
  abmax=np.max([np.nanmax(xdata[2]),np.nanmax(xdata[3])])


In [21]:
for cast in sorted(os.listdir(sample_data_dir)):
    if cast.endswith('updated.nc'):
        cruise_data_nc = xa.load_dataset(sample_data_dir+cast)
        ctd_df = cruise_data_nc.to_dataframe()
        
        sbe_p = sbe_ctd_plots.CTDProfilePlot()
        plt,fig =sbe_p.plot3var(varname=['par','','turbidity','','chlor_fluorescence',''],
                          xdata=[ctd_df.par,np.array([]),ctd_df.turbidity,np.array([]),ctd_df.chlor_fluorescence,np.array([])],
                          ydata=ctd_df.index.get_level_values('depth'),
                          secondary=True,
                          xlabel=['PAR','Fluor','Turbidity'])

        DefaultSize = fig.get_size_inches()
        fig.set_size_inches( (DefaultSize[0], DefaultSize[1]*3) )
        plt.savefig(sample_data_dir+cast.replace('.nc','_ParTurbFluor.png'))
        plt.close(fig)